# BDCC project - Loader Cloud Function development

**[Big Data and Cloud Computing](https://www.dcc.fc.up.pt/~edrdo/aulas/bdcc), DCC/FCUP**

Make sure you go through the __[Google Cloud Functions Pub/Sub tutorial](https://cloud.google.com/functions/docs/tutorials/pubsub)__ before you start developing the LCF.



## GCP authentication function

In [0]:
PROJECT_ID = 'bdcc20-p1'  # TODO change to your project id

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# The authentication method 
def google_colab_authenticate(projectId, keyFile=None, debug=True):  
    import os
    from google.colab import auth
    if keyFile == None:
      keyFile='/content/bdcc-colab.json'
    if os.access(keyFile,os.R_OK):
      if debug:
        print('Using key file "%s"' % keyFile)
      os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '%s' % keyFile
      os.environ['GCP_PROJECT'] = projectId 
      os.environ['GCP_ACCOUNT'] = 'bdcc-colab@' + projectId + '.iam.gserviceaccount.com'
      !gcloud auth activate-service-account --key-file="$GOOGLE_APPLICATION_CREDENTIALS" --project="$GCP_PROJECT"
    else:
      if debug:
        print('No key file given. You may be redirected to the verification code procedure.')
      auth.authenticate_user()
      !gcloud config set project $projectId
    !gcloud info | grep -e Account -e Project

# Copy key file from Google Drive if available 
# to a path without spaces (it usually creates problems)
!test -f "/content/drive/My Drive/bdcc-colab.json" && cp "/content/drive/My Drive/bdcc-colab.json" /content/bdcc-colab.json
google_colab_authenticate(PROJECT_ID)


Using key file "/content/bdcc-colab.json"
Activated service account credentials for: [bdcc-cloud@bdcc20-p1.iam.gserviceaccount.com]
Account: [bdcc-cloud@bdcc20-p1.iam.gserviceaccount.com]
Project: [bdcc20-p1]


In [4]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/My Drive/bdcc-colab.json' 
!echo $GOOGLE_APPLICATION_CREDENTIALS

/content/drive/My Drive/bdcc-colab.json


In [5]:
from google.cloud import storage

storage_client = storage.Client()
buckets = storage_client.list_buckets()
print('-- List of buckets in project \"' + storage_client.project + '\"')

for b in buckets:
  print(b.name)


-- List of buckets in project "bdcc20-p1"
bdcc20-movie_data


In [6]:
# To enable the GPU access Edit > Notebook settings and set the Hardware accelerator to GPU.

%tensorflow_version 2.x 
import tensorflow as tf

print("GPU device: " + tf.test.gpu_device_name())

from tensorflow.python.client import device_lib

tf_devices = device_lib.list_local_devices()

for x in tf_devices:
  print('------')
  print(x)

GPU device: 
------
name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8945757363232109252

------
name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7364822986433981580
physical_device_desc: "device: XLA_CPU device"



## Cloud function code

This should be placed in a single cell to facilitate cloud function.

Note that you cannot use "magic" notebook extensions such as `! shell command` or `%%bigquery`.

In [7]:
# Imports
import base64
import pandas as pd
import os
import tempfile
import time
from zipfile import ZipFile
import google.cloud.bigquery as bq
import google.cloud.storage as gcs


# Parameters
PROJECT_ID = 'bdcc20-p1'  # TODO change to your project id
BUCKET_NAME = 'bdcc20-movie_data' # TODO change to your bucket name
PUBSUB_TOPIC = 'dispatcher' 
SUB_DIRECTORY = 'bdcc1920_project_outputs'
OUTPUT_ZIP_FILE = 'output.zip'

DEBUG = True 
RUNNING_IN_COLAB = os.environ.get('COLAB_GPU') != None

TMP_DIR=tempfile.mkdtemp(prefix='LCF_')

# Debug method
def debug(message):
  if DEBUG:
     print(message)

# Authenticate to GCP if running in Colab
if RUNNING_IN_COLAB:
  google_colab_authenticate(PROJECT_ID)

# Initialize interface to BigQuery and GCS
BQ_CLIENT = bq.Client(PROJECT_ID)
GCS_CLIENT = gcs.Client(PROJECT_ID)
BUCKET = gcs.Bucket(GCS_CLIENT, BUCKET_NAME)

def get_data_from_cloud_storage(dataset_id):
  bucket_path = '%s/%s/%s' % (SUB_DIRECTORY, dataset_id, OUTPUT_ZIP_FILE)
  local_zip_file = '%s/%s' % (TMP_DIR, OUTPUT_ZIP_FILE)
  debug('Downloading gs://%s/%s to %s' % (BUCKET_NAME,bucket_path,local_zip_file))
  blob = gcs.Blob(bucket_path, BUCKET)
  with open(local_zip_file, 'wb') as out:
    blob.download_to_file(out)
  

def unzip_data_file():
  local_zip_file = '%s/%s' % (TMP_DIR, OUTPUT_ZIP_FILE)
  debug('Unzipping %s' % local_zip_file)
  with ZipFile(local_zip_file) as zf:
    zf.extractall(TMP_DIR)
  debug('Unzipping done')

def load_movie_agg_data(dataset_id):
  tid = 'movies_agg'
  table_name = '%s.%s.%s' % (PROJECT_ID, dataset_id, tid)
  
  # Read parquet file
  parquet_files_path = '%s/%s.parquet' % (TMP_DIR, tid)
  debug('Reading Parquet files from %s' % parquet_files_path)
  pdf = pd.read_parquet(parquet_files_path)
  debug(str(pdf.head(5)))


  # Create BigQuery table
  table = bq.Table(table_name)
  
  table.schema = (
        bq.SchemaField("movieId", "INTEGER", "REQUIRED"),
        bq.SchemaField("title",  "STRING", "REQUIRED"),
        bq.SchemaField("year", "INTEGER", "REQUIRED"),
        bq.SchemaField("imdbId", "INTEGER", "REQUIRED"),
        bq.SchemaField("numRatings", "INTEGER", "REQUIRED"),
        bq.SchemaField("avgRating", "FLOAT", "REQUIRED"),
  )
  debug('Creating %s' % table_name)
  BQ_CLIENT.create_table(table)

  debug('Populating %s with %d rows' % (table_name, len(pdf)))
  load_job = BQ_CLIENT.load_table_from_dataframe(pdf, table)

  while load_job.running():
     debug('waiting for load job to complete')
     time.sleep(1)

  debug('Done with table %s' % table_name)

def load_tfidf_data(dataset_id):
  tid = 'tfidf'
  table_name = '%s.%s.%s' % (PROJECT_ID, dataset_id, tid)
  
  # Read parquet file
  parquet_files_path = '%s/%s.parquet' % (TMP_DIR, tid)
  debug('Reading Parquet files from %s' % parquet_files_path)
  pdf = pd.read_parquet(parquet_files_path)
  debug(str(pdf.head(5)))


  # Create BigQuery table
  table = bq.Table(table_name)
  
  table.schema = (
        bq.SchemaField("word", "STRING", "REQUIRED"),
        bq.SchemaField("movieId",  "INTEGER", "REQUIRED"),
        bq.SchemaField("tfidf", "FLOAT", "REQUIRED"),
  )
  debug('Creating %s' % table_name)
  BQ_CLIENT.create_table(table)

  debug('Populating %s with %d rows' % (table_name, len(pdf)))
  load_job = BQ_CLIENT.load_table_from_dataframe(pdf, table)

  while load_job.running():
     debug('waiting for load job to complete')
     time.sleep(1)

  debug('Done with table %s' % table_name)


def load_jaccard_index_data(dataset_id):
  tid = 'jaccardIndex'
  table_name = '%s.%s.%s' % (PROJECT_ID, dataset_id, tid)
  
  # Read parquet file
  parquet_files_path = '%s/%s.parquet' % (TMP_DIR, tid)
  debug('Reading Parquet files from %s' % parquet_files_path)
  pdf = pd.read_parquet(parquet_files_path)
  debug(str(pdf.head(5)))


  # Create BigQuery table
  table = bq.Table(table_name)
  
  table.schema = (
        bq.SchemaField("movie1", "INTEGER", "REQUIRED"),
        bq.SchemaField("movie2",  "INTEGER", "REQUIRED"),
        bq.SchemaField("user", "INTEGER", "REQUIRED"),
        bq.SchemaField("index", "INTEGER", "REQUIRED"),
        bq.SchemaField("jaccard_index", "FLOAT", "REQUIRED"),
  )
  debug('Creating %s' % table_name)
  BQ_CLIENT.create_table(table)

  debug('Populating %s with %d rows' % (table_name, len(pdf)))
  load_job = BQ_CLIENT.load_table_from_dataframe(pdf, table)

  while load_job.running():
     debug('waiting for load job to complete')
     time.sleep(1)

  debug('Done with table %s' % table_name)

def handle_pubsub_message(event, context):
  debug('Event: %s' % event)
  debug('Context: %s' % context)

  if RUNNING_IN_COLAB:
    dataset_id = event['data']
  else:
    dataset_id =  base64.b64decode(event['data']).decode('utf-8')
  
  debug('Dataset: %s' % dataset_id)
  
  get_data_from_cloud_storage(dataset_id)
  unzip_data_file()
  print(dataset_id)

  debug('Deleting previous BiqQuery dataset (if any)')

  print(dataset_id)
  BQ_CLIENT.delete_dataset(dataset_id, delete_contents = True, not_found_ok = True)
  BQ_CLIENT.create_dataset(dataset_id)

  debug('Created BiqQuery dataset')

  load_movie_agg_data(dataset_id)
  load_tfidf_data(dataset_id)
  load_jaccard_index_data(dataset_id) # bonus
  debug('Done for data set %s' % dataset_id)


Using key file "/content/bdcc-colab.json"
Activated service account credentials for: [bdcc-cloud@bdcc20-p1.iam.gserviceaccount.com]
Account: [bdcc-cloud@bdcc20-p1.iam.gserviceaccount.com]
Project: [bdcc20-p1]


## Test cloud function locally

In [8]:
!gsutil ls gs://bdcc20-movie_data/bdcc1920_project_outputs/medium1

gs://bdcc20-movie_data/bdcc1920_project_outputs/medium1/output.zip
gs://bdcc20-movie_data/bdcc1920_project_outputs/medium1/jaccardIndex.parquet/
gs://bdcc20-movie_data/bdcc1920_project_outputs/medium1/movies_agg.parquet/
gs://bdcc20-movie_data/bdcc1920_project_outputs/medium1/tfidf.parquet/


In [9]:
BUCKET = gcs.Bucket(GCS_CLIENT, BUCKET_NAME)
list_of_blobs = BUCKET.list_blobs(prefix=SUB_DIRECTORY)
for blob in list_of_blobs:
  print('==> ' + blob.id)

==> bdcc20-movie_data/bdcc1920_project_outputs/medium1/jaccardIndex.parquet/._SUCCESS.crc/1586251143340792
==> bdcc20-movie_data/bdcc1920_project_outputs/medium1/jaccardIndex.parquet/.part-00000-9ae7e81c-7872-462b-b758-fe9bc79b7a5d-c000.snappy.parquet.crc/1586251137880750
==> bdcc20-movie_data/bdcc1920_project_outputs/medium1/jaccardIndex.parquet/.part-00001-9ae7e81c-7872-462b-b758-fe9bc79b7a5d-c000.snappy.parquet.crc/1586251139861421
==> bdcc20-movie_data/bdcc1920_project_outputs/medium1/jaccardIndex.parquet/.part-00002-9ae7e81c-7872-462b-b758-fe9bc79b7a5d-c000.snappy.parquet.crc/1586251138733352
==> bdcc20-movie_data/bdcc1920_project_outputs/medium1/jaccardIndex.parquet/.part-00003-9ae7e81c-7872-462b-b758-fe9bc79b7a5d-c000.snappy.parquet.crc/1586251134717107
==> bdcc20-movie_data/bdcc1920_project_outputs/medium1/jaccardIndex.parquet/.part-00004-9ae7e81c-7872-462b-b758-fe9bc79b7a5d-c000.snappy.parquet.crc/1586251137331425
==> bdcc20-movie_data/bdcc1920_project_outputs/medium1/jaccardI

In [10]:
dataset = 'medium1' #@param ["tiny1", "tiny2", "tiny3", "tiny4", "medium1", "medium2", "medium3", "medium4", "large1", "large2", "large3", "large4", "large5"] {allow-input: true}
handle_pubsub_message({ 'data': dataset}, None)

Event: {'data': 'medium1'}
Context: None
Dataset: medium1
Unzipping /tmp/LCF_fhfsm2p4/output.zip
Unzipping done
medium1
Deleting previous BiqQuery dataset (if any)
medium1
Created BiqQuery dataset
Reading Parquet files from /tmp/LCF_fhfsm2p4/movies_agg.parquet
   movieId  ... avgRating
0       24  ...  3.179306
1      888  ...  2.319775
2      944  ...  3.819965
3     1102  ...  2.610465
4     1176  ...  3.889452

[5 rows x 6 columns]
Creating bdcc20-p1.medium1.movies_agg
Populating bdcc20-p1.medium1.movies_agg with 328 rows
waiting for load job to complete
waiting for load job to complete
waiting for load job to complete
waiting for load job to complete
waiting for load job to complete
Done with table bdcc20-p1.medium1.movies_agg
Reading Parquet files from /tmp/LCF_fhfsm2p4/tfidf.parquet
               word  movieId     tfidf
0   dying and death    48043  0.082040
1  aftermath of war     1939  1.344348
2          hostages     2058  1.440373
3         dystopian    38881  8.066089
4   p

## Trigger cloud function once it is deployed

Before deployment do not forget to add the following dependencies to __REQUIREMENTS.txt__ in the function definitions (note that __pyarrow__ is required for Parquet data handling using Pandas):

```
pyarrow 
pandas
google.cloud.bigquery
google.cloud.storage
```

In [11]:
dataset = 'medium1' #@param ["tiny1", "tiny2", "tiny3", "tiny4", "medium1", "medium2", "medium3", "medium4", "large1", "large2", "large3", "large4", "large5"] {allow-input: true}
!gcloud pubsub topics publish "$PUBSUB_TOPIC" --message "$dataset"

messageIds:
- '1095662293375688'


In [0]:
# TODO You may now check, as in previous notebooks:
# - inspect function logs
# - your BigQuery data is ok with some queries in the notebook and/or in the BigQuery Web UI

In [13]:
!gcloud functions logs read loader_cloud_function 

LEVEL  NAME                   EXECUTION_ID      TIME_UTC                 LOG
I      loader_cloud_function  1095649122874450  2020-04-07 09:19:36.453  waiting for load job to complete
I      loader_cloud_function  1095649122874450  2020-04-07 09:19:37.493  waiting for load job to complete
I      loader_cloud_function  1095649122874450  2020-04-07 09:19:38.566  Done with table bdcc20-p1.medium1.tfidf
I      loader_cloud_function  1095649122874450  2020-04-07 09:19:38.566  Reading Parquet files from /tmp/LCF_6jysbvd3/jaccardIndex.parquet
I      loader_cloud_function  1095649122874450  2020-04-07 09:19:39.599     movie1  movie2   user  index  jaccard_index
I      loader_cloud_function  1095649122874450  2020-04-07 09:19:39.599  0    2890    3052  16063   1881       0.117101
I      loader_cloud_function  1095649122874450  2020-04-07 09:19:39.599  1    4720   68157  22114   1860       0.084110
I      loader_cloud_function  1095649122874450  2020-04-07 09:19:39.599  2    3052   68157  24985  

In [24]:
cloudFunctionName = 'loader_cloud_function'
!gcloud functions logs read $cloudFunctionName --limit 1000 

LEVEL  NAME                   EXECUTION_ID      TIME_UTC                 LOG
D      loader_cloud_function  1087417895409978  2020-03-31 13:44:02.999  Function execution started
I      loader_cloud_function  1087417895409978  2020-03-31 13:44:08.908  Event: {'@type': 'type.googleapis.com/google.pubsub.v1.PubsubMessage', 'attributes': None, 'data': 'dGlueTE='}
I      loader_cloud_function  1087417895409978  2020-03-31 13:44:08.908  Context: {event_id: 1087417895409978, timestamp: 2020-03-31T13:43:55.064Z, event_type: google.pubsub.topic.publish, resource: {'service': 'pubsub.googleapis.com', 'name': 'projects/bdcc20-p1/topics/dispatcher', 'type': 'type.googleapis.com/google.pubsub.v1.PubsubMessage'}}
I      loader_cloud_function  1087417895409978  2020-03-31 13:44:08.908  Dataset: tiny1
I      loader_cloud_function  1087417895409978  2020-03-31 13:44:08.908  Downloading gs://bdcc1920_project_outputs/tiny1/output.zip to /tmp/LCF_9hne5g8_/output.zip
I      loader_cloud_function  1087417895

In [33]:
# %%bigquery "magics". pandas dataframe
%%bigquery --project $PROJECT_ID
SELECT *
FROM `bdcc20-p1.tiny1.tfidf`
LIMIT 10

'''
# dataframe objects
%%bigquery --project $PROJECT_ID tfidf_df
SELECT *
FROM `bdcc20-p1.tiny1.tfidf`
LIMIT 10
'''


Executing query with job ID: 788f30ab-158a-4463-9c49-c0e2dffe139a
Query executing: 0.20s


ERROR:
 400 Syntax error: Unexpected string literal '''\n# dataframe objects\n%%bigquery --project ...''' at [5:1]

(job ID: 788f30ab-158a-4463-9c49-c0e2dffe139a)

       -----Query Job SQL Follows-----        

    |    .    |    .    |    .    |    .    |
   1:SELECT *
   2:FROM `bdcc20-p1.tiny1.tfidf`
   3:LIMIT 10
   4:
   5:'''
   6:# dataframe objects
   7:%%bigquery --project $PROJECT_ID tfidf_df
   8:SELECT *
   9:FROM `bdcc20-p1.tiny1.tfidf`
  10:LIMIT 10
  11:'''
    |    .    |    .    |    .    |    .    |


In [34]:
# %%bigquery "magics". pandas dataframe
%%bigquery --project $PROJECT_ID
SELECT *
FROM `bdcc20-p1.medium1.tfidf`
LIMIT 10

'''
# dataframe objects
%%bigquery --project $PROJECT_ID tfidf_df
SELECT *
FROM `bdcc20-p1.medium1.tfidf`
LIMIT 10
'''


Executing query with job ID: 232f4751-ae32-444d-bfb7-e466d9dde396
Query executing: 0.24s


ERROR:
 400 Syntax error: Unexpected string literal '''\n# dataframe objects\n%%bigquery --project ...''' at [5:1]

(job ID: 232f4751-ae32-444d-bfb7-e466d9dde396)

       -----Query Job SQL Follows-----        

    |    .    |    .    |    .    |    .    |
   1:SELECT *
   2:FROM `bdcc20-p1.medium1.tfidf`
   3:LIMIT 10
   4:
   5:'''
   6:# dataframe objects
   7:%%bigquery --project $PROJECT_ID tfidf_df
   8:SELECT *
   9:FROM `bdcc20-p1.medium1.tfidf`
  10:LIMIT 10
  11:'''
    |    .    |    .    |    .    |    .    |


In [0]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

In [36]:
for d_info in client.list_datasets():
   print('Dataset ' + d_info.full_dataset_id)

Dataset bdcc20-p1:medium1
Dataset bdcc20-p1:tiny1


In [37]:
madf = client.query(
  '''
  SELECT *
  FROM `bdcc20-p1.tiny1.tfidf`
  LIMIT 10
  '''
)
madf.to_dataframe()

,word,movieId,tfidf
0,pixar,1,2.321928
1,game,2,2.321928
2,fantasy,2,2.321928
3,Robin Williams,2,2.321928
4,magic board game,2,2.321928
5,moldy,3,2.321928
6,old,3,2.321928
7,pregnancy,5,2.321928
8,fun,1,1.160964
9,remake,5,1.321928


In [38]:
madf = client.query(
  '''
  SELECT *
  FROM `bdcc20-p1.medium1.tfidf`
  LIMIT 10
  '''
)
madf.to_dataframe()

,word,movieId,tfidf
0,strong female lead,3408,2.062177
1,unique,26326,2.062177
2,Comedy,95441,1.959410
3,predictable,95441,3.473214
4,revenge,68157,0.818961
5,disease,48043,1.021024
6,alternate history,68157,2.285392
7,twist ending,1687,0.783018
8,Diane Kruger,68157,0.761797
9,anti-war,2890,7.617973


In [39]:
madf = client.query(
  '''
  SELECT *
  FROM `bdcc20-p1.tiny1.jaccardIndex`
  LIMIT 10
  '''
)
madf.to_dataframe()

,movie1,movie2,user,index,jaccard_index
0,1,6,189,27,0.142857
1,1,2,176,21,0.119318
2,1,10,187,19,0.101604
3,1,3,154,11,0.071429
4,1,5,152,7,0.046053
5,1,7,160,7,0.043750
6,2,10,100,9,0.090000
7,2,6,111,8,0.072072
8,2,7,64,6,0.093750
9,2,3,63,5,0.079365


In [40]:
madf = client.query(
  '''
  SELECT *
  FROM `bdcc20-p1.medium1.jaccardIndex`
  LIMIT 10
  '''
)
madf.to_dataframe()

,movie1,movie2,user,index,jaccard_index
0,95441,126430,1958,46,0.023493
1,126430,140737,539,16,0.029685
2,95441,96367,1689,13,0.007697
3,2058,3052,14767,1110,0.075168
4,2058,3408,14461,1096,0.075790
5,2058,2890,11640,1019,0.087543
6,2058,4720,12326,739,0.059955
7,2058,5502,9820,658,0.067006
8,2058,68157,20860,641,0.030729
9,2058,2447,6154,238,0.038674
